In [1]:
#%matplotlib inline
import joblib
import csv, gzip, os, matplotlib.pyplot as plt, numpy as np, pandas as pd, pysam, scipy, sys, joblib, seaborn as sns#; sns.set()
from glob import glob
from collections import Counter
from scipy.stats import stats
from datetime import datetime
from multiprocessing import Pool

%load_ext autoreload
%autoreload 2
# import modules from src dir
sys.path.insert(0, "/home/lpryszcz/src/nanoRMS3/src")
#import eif_new as iso_new
from predict_mods import bam2data, load_data, load_data_single, get_KS, get_revcomp, \
    get_calls, get_regions, get_covered_regions, get_significant_mers, \
    get_classifier, get_freq, GradientBoostingClassifier, random_state
# make sure plots are inline (modPhred uses agg)
%matplotlib inline

In [4]:
%%time
import itertools
boi = "T" # pseudoU
n = 1 # n=3 for 7-mer; n=1 for 3-mer; n2 for 5-mer
k = 2*n+1
bases='ACGT'
mers = ["".join(m) for m in itertools.product(set(bases), repeat=k)]; len(mers)
mers = [m for m in mers if m[n]==boi]; mers.sort(); len(mers)
#mers = ["A"*n+"T"+"A"*n, ]

# initially get only most interesting mers using 3 features and ignoring neighbours
nn = 1 # how many neighbour positions to take into account
dt_shift = 10 # expected shift between center of the pore and motor protein in bases
minReads = 10 # min number of reads (per strand) to keep position
# define features
#features = ["si", "tr"] # TR needed here
features = ["si", "mp", "dt0", "dt%s"%dt_shift] + ["t%s"%b for b in "ACGT"] # here we don't need TR as TA/C/G/T will capture it all right
feature_names = ["%s_%s"%(f.upper(), i) for f in features for i in range(-nn, nn+1)]

threads=6; rna=True
fasta = "/home/lpryszcz/cluster/rna_mods/RMaP_challenge/Manja_Merz/3a/reference/psU_template.fa" # reference FastA
regions = get_regions(fasta)

samples = ("unmod", "mod")
bams = ["/home/lpryszcz/cluster/rna_mods/RMaP_challenge/Manja_Merz/3a/fast5/training_control.bam", 
        "/home/lpryszcz/cluster/rna_mods/RMaP_challenge/Manja_Merz/3a/fast5/training_mod.bam"
        ]
mer2data = load_data_single(fasta, bams, features, nn, rna, regions, minReads, maxReads=50000, mers=mers, n=n, 
                            max_frac_mod_per_read=.3, only_forward=True)
len(mer2data), mer2data.keys()

CPU times: user 1min 51s, sys: 1.85 s, total: 1min 53s
Wall time: 1min 53s


(9, dict_keys(['ATA', 'GTA', 'CTC', 'ATG', 'GTG', 'CTA', 'ATC', 'GTC', 'CTG']))

In [3]:
%%time
from sklearn.ensemble import HistGradientBoostingClassifier

mer2clf = {}
for mer in sorted(mer2data.keys()):
    X = np.vstack(mer2data[mer][:2])#; print(mer, X.shape)
    y = np.zeros(len(X), dtype="int")
    y[len(X)//2:] = 1
    # use every 10th example for testing
    sel = np.zeros(len(X), dtype="bool")
    sel[::10] = True
    clf = HistGradientBoostingClassifier().fit(X[~sel], y[~sel])
    y_pred = clf.predict(X[sel])
    clf.nn = nn
    clf.acc = np.mean(y_pred == y[sel])
    clf.features = features
    clf.feature_names = feature_names
    mer2clf[mer] = clf
    clf.features = features
    print("%s\t%s\t%.3f"%(mer, len(y[~sel]), clf.acc))

acc =np.array([clf.acc for mer, clf in mer2clf.items()])
print("%.3f\t%.3f"%(np.median(acc), acc.mean()))

modelsfn = "3a.models.%s.%s_mers.%s_features.lzma"%(boi, k, len(feature_names))
joblib.dump(mer2clf, modelsfn)
# 0.840 vs 0.801 GBC ATA 18s vs 16:47
# 0.898 vs 0.873 GBC AATAA 2s vs 1:23
# 0.890 vs 0.870 GBC AAATAAA

ATA	2361663	0.841
ATC	2410369	0.831
ATG	2412990	0.894
CTA	2411757	0.873
CTC	2345115	0.859
CTG	2400348	0.829
GTA	2396206	0.883
GTC	2411929	0.889
GTG	2386573	0.864
0.864	0.862
CPU times: user 21min 30s, sys: 1.45 s, total: 21min 32s
Wall time: 2min 41s


['3a.models.T.3_mers.24_features.lzma']

In [4]:
%timeit clf.predict(X[:1]) # HistGBC 1.61 ms ± 53.2 µs

1.72 ms ± 75.7 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [5]:
%%time 
clf = GradientBoostingClassifier().fit(X[~sel], y[~sel])
y_pred = clf.predict(X[sel])
np.mean(y_pred == y[sel])
%timeit clf.predict(X[:1]) # GBC 153 µs ± 619 ns

CPU times: user 16min 47s, sys: 225 ms, total: 16min 47s
Wall time: 16min 47s


0.8019488809368652

In [7]:
%%time
%timeit clf.predict(X[:1]) # GBC 153 µs ± 619 ns

148 µs ± 594 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
CPU times: user 12 s, sys: 14 µs, total: 12 s
Wall time: 12 s
